In [1]:
import numpy as np
import csv
import pandas as pd
import time
import matplotlib.pyplot as plt

from sklearn import preprocessing

from sklearn.cluster import KMeans
#from sklearn.mixture import GMM # gaussian mixture model
## ultimately: try LDA?

In [2]:
# Predict via the user-specific median.
# If the user has no data, use the global median.

train_file = 'train.csv'
test_file  = 'test.csv'
soln_file  = 'user_median.csv'
prof_file = 'profiles.csv'
art_file = 'artists.csv'

In [3]:
# Load the training data.

# new: partition into training and validation set
# if we want to use all data for training? just set rMax to > 5M
rMax = 6000000 # use first 3M to populate training matrix; after that, test
r = 0
idx = 1

train_data = {}
test_data = {}
with open(train_file, 'r') as train_fh:
    train_csv = csv.reader(train_fh, delimiter=',', quotechar='"')
    next(train_csv, None)

    for row in train_csv:
        user   = row[0]
        artist = row[1]
        plays  = row[2]
        
        if r<rMax: # then populate training data
            if not user in train_data:
                train_data[user] = {}
        
            train_data[user][artist] = int(plays)
            r = r+1
        else: # else populate validation set
            test_data[idx] = [user, artist, plays] # with valid set: now how extra entry for real # of plays
            idx = idx+1
    

In [5]:
nTest = idx-1

In [6]:
# Alternatively: Load the 'real' test data.
test_data = {}
with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)
    for row in test_csv:
        id   = int(row[0])
        user = row[1]
        artist  = row[2]
    
        #if not user in train_data:
        #    train_data[user] = {}
        
        test_data[id] = [user, artist]

In [7]:
nTest = int(id)
tmp = test_data[str(10)]
print prof_idx[tmp[0]]
print art_idx[tmp[1]]

KeyError: '10'

In [8]:
# Load the user profile data into dictionaries
prof_data = {}
prof_sex = {}
prof_age = {}
prof_loc = {}
prof_idx = {}
idx = 0
with open(prof_file, 'r') as prof_fh:
    prof_csv = csv.reader(prof_fh, delimiter=',', quotechar='"')
    next(prof_csv, None)
    for row in prof_csv:
        user   = row[0]
        sex = row[1]
        age = row[2]
        loc = row[3]
        # add other
    
        if not user in prof_data:
            prof_data[user] = {}
        
        #train_data[user][artist] = int(plays)
        prof_data[user] = [sex, age, loc]
        # alternatively: store in separate dictionaries?
        prof_sex[user] = sex
        prof_age[user] = age
        prof_loc[user] = loc
        prof_idx[user] = idx
        idx = idx+1

In [9]:
# also, for later: create artist dictionary
# saves time from using find commands for writing data
art_data = {}
art_name = {}
art_idx = {}
idx = 0
with open(art_file, 'r') as art_fh:
    art_csv = csv.reader(art_fh, delimiter=',', quotechar='"')
    next(art_csv, None)
    for row in art_csv:
        artist   = row[0]
        name = row[1]
        
        #train_data[user][artist] = int(plays)
        art_data[artist] = [name, idx]
        # alternatively: store in separate dictionaries?
        art_name[artist] = name
        art_idx[artist] = idx
        idx = idx+1


In [10]:
df_train = pd.read_csv(train_file)
df_test= pd.read_csv(test_file) # etc if want to handle as pandas df's

In [11]:
df_prof = pd.read_csv(prof_file)
prof_vals = df_prof.values
all_ids = prof_vals.T[:][0]
print all_ids.shape
print all_ids[0]


(233286,)
fa40b43298ba3f8aa52e8e8863faf2e2171e0b5d


In [12]:
df_arts = pd.read_csv(art_file)
art_vals = df_arts.values
all_arts = art_vals.T[:][0]
print all_arts.shape
print all_arts[0]

(2000,)
03098741-08b3-4dd7-b3f6-1b0bfa2c879c


In [13]:
# now: PCA analysis
# want to construct matrix where row is one user, columns are artists, entries are listens
# so, matrix will be 233286 x 2000

s1 = all_ids.shape[0]
s2 = all_arts.shape[0]

data_mat = np.ndarray([s1, s2])
print data_mat.shape

(233286, 2000)


In [14]:
# instead: populate using dictionaries -> should be much quicker? no lookups

t = time.time()
# data_mat = np.ndarray([s1, s2]) # this was defined above
# for each user-artist combo
for i in range(s1):
    for j in range(s2):
        usr = all_ids[i]
        art = all_arts[j]
        #try-catch in case no data
        try:
            tmp = train_data[usr][art]
            #print i,',',j
            #print tmp
        except KeyError: # if no corresponding key
            tmp = 0
        # populate matrix
        data_mat[i][j]=tmp
print 'elapsed time: ', time.time()-t
print 'All done!'
        

elapsed time:  829.774206877
All done!


In [15]:
print data_mat.shape

(233286, 2000)


In [16]:
# define script for file writing
def write_to_file(filename, data):
    with open(filename, "w") as f:
        #f.write("Id,Prediction\n")
        for i in range(s1): # for each row
            for j in range(s2): # for each column
                f.write(str(data[i][j]) + ",") # write matrix entry
            f.write("\n") # new line after each row

In [17]:
## output matrix as a CSV file
write_to_file("usr-art_mat.csv", data_mat)
#data_mat.tofile('usr-art_mat.csv',sep=',')

In [19]:
# clear space in case we've been going back and forth?
# del L, V, data_proj, Sigma
del km

In [20]:
# standardize data? before or after finding principal components?
# del data_sub # for memory conservation
t = time.time()
std_scale = preprocessing.StandardScaler().fit(data_mat)
data_scaled = std_scale.transform(data_mat)
print time.time()-t

278.563163996


In [21]:
# and use this to find covariance, and thus principal components?
# Sigma = np.cov(data_mat) # PCA on raw data
# Sigma = np.cov(data_sub.T) # PCA on mean-subtracted data
Sigma = np.cov(data_scaled.T)
print Sigma.shape


(2000, 2000)


In [22]:
# find principal components
L, V = np.linalg.eig(Sigma) # L holds eigenvalues, V holds eigenvectors

In [23]:
# project onto PCA axes
data_proj = np.dot(data_mat,V)
print data_proj.shape

(233286, 2000)


In [14]:
for u in range(10): # plot first 10 users in PCA basis?
    plt.plot(data_proj[u])
plt.show()

In [24]:
# simplest clustering: KMeans
nPCs = 10 # how many PCs to include in fit
K = 15 # how many cluster centers to fit

# if validating: set aside users to compare predictions to?
nValid = 10000 # set aside 10k

data_tofit = data_proj.T[:nPCs].T
data_tofit = data_tofit[nValid:] # leave out validation set


# construct classifier
km = KMeans(n_clusters=K)
# train?
t = time.time()
km.fit(data_proj.T[:nPCs].T) # use all users
#km.fit(data_tofit) # if validating, fit a subset
print 'Finished training!'
print 'training time = ', time.time()-t,' seconds'

Finished training!
training time =  51.0127370358  seconds


In [25]:
# get predictions?
# first, try on a test sample:

tmp = data_proj.T[:nPCs].T
# tmp = tmp[:nValid] # in not validating, want to predict all clusters
print tmp.shape

# get cluster centers
mus = km.cluster_centers_

# returns cluster center
predKs = km.predict(tmp)
print predKs[:100]

(233286, 10)
[ 1  5  1 13  1  5  1  1  1  1  5  3  1  1  1  1  5  1  5  1  1 11  1  1  5
  1 11  1  1  1  1  1  1  1 13  1  5  1  1 10  1  1 10  5  1  1  1  1  1  1
  5  5  1  1  1  4  1  1  1  1  1  1  5  1 11  1  1  1  1  1  1  1  5  1  1
  1  1  1  1 13  1  1  1  1  1  2  1  1  1  1  1  0  5  1  1  1  1  1  1  1]


In [ ]:
%matplotlib inline

In [ ]:
# check if at all reasonable?
plt.plot(mus[0])
plt.plot(tmp[0])
plt.show()

In [118]:
# debugging to get cluster median sorting correct
tmp = np.equal(predKs, 4)
idx = np.nonzero(tmp)[0]
#print idx
k_ids = all_ids[idx]
print k_ids.shape
print K

(641L,)
15


In [122]:
# find medians *of each artist* within each cluster
clusterMem = {} # dictionary of members for each user
for k in range(K): # for each cluster
    tmp = np.equal(predKs,k)
    k_idx = np.nonzero(tmp)[0] # numeric indices of all users assigned to that cluster
    k_ids = all_ids[k_idx] # correspondeng alphanumeric user ids
    clusterMem[k] = k_ids
    

In [124]:
print clusterMem[0].shape
print clusterMem[10].shape

(34948L,)
(771L,)


In [159]:
# to predict? first play with main loop
id = nTest
[usr, art] = test_data[id]
print usr, art
# find user index
i = prof_idx[usr]
# find assigned cluster
k = predKs[i]
# construct list of plays of artist 'art' in cluster 'K'
playslist = []
for u in k_ids: # for all users in cluster k
    # try for KeyError exception if no data
    try:
        playslist.append(train_data[u][art]) # see if this cluster user has listened to artist
    except KeyError: # if not
        # playslist.append(0) # can add zero?
        pass # or just pass?
out = np.median(playslist) # take median plays for that artist within this cluster
print playslist
print out
print float(np.sum(playslist)) / clusterMem[k].shape[0]

de29005de66c93b31a34991b0c72d9f70b9fc313 c0b2500e-0cef-4130-869d-732b23ed9df5
[9334, 570, 788, 173, 907, 10182, 3875, 192, 4133, 1183, 6412, 588, 638, 10941, 9409, 22228, 770, 7769, 329, 744, 360, 3062, 2209, 220, 279, 1071, 829, 1306, 1776, 1639, 1193, 269, 658, 904, 327, 2758, 32391, 901, 696, 7868, 4698, 11540, 677, 162, 867, 265, 2968, 488, 11012, 410, 3178, 4049, 457, 2612, 106, 879, 12312, 2047, 636, 2092, 2179, 6794, 140, 1570, 3610, 12580, 7124, 189, 1033, 3905, 1880, 2240, 541, 392, 100, 448, 5186, 2640, 17044, 749, 996, 628, 572, 432, 10299, 6301, 153, 1409, 647, 2772, 544, 1882, 511, 13433, 767, 676, 652, 153, 3167, 170, 184, 1057, 7559, 4064, 1805, 243, 12584, 4676, 1363]
1071.0
2.15270644131


In [157]:
# seems to do reasonably well, and not not take forever
# try full prediciton set?

pred_fin = np.zeros([nTest,1])
pred_errs = np.zeros([nTest,1]) # also store errors if validating

for itmp in range(nTest): # for all test combos
    idx = itmp + 1 # count from 1 to nTest, rather than 0 to 1-nTest
    
    # [usr, art] = test_data[art] # retrieve user-artist pairs for testing
    
    # if validating? retrieve extra value which has true number of plays
    [usr, art, tru] = test_data[art]
    
    # find user index
    i = prof_idx[usr]
    # find assigned cluster
    k = predKs[i]
    # construct list of plays of artist 'art' in cluster 'K'
    playslist = []
    for u in k_ids: # for all users in cluster k
        # try for KeyError exception if no data
        try:
            playslist.append(train_data[u][art]) # see if this cluster user has listened to artist
        except KeyError: # if not
            # playslist.append(0) # can add zero?
            pass # or just pass?
    out = np.median(playslist) # take median plays for that artist within this cluster
    pred_fin[itmp] = out # populate predicitons
    pred_errs[itmp]= np.absolute(out - true) # take absolute value of error
    

4154804


In [ ]:
# error metric? average absolute error
MAE = np.mean(pred_errs)
print MAE # target: 137 for user medians

In [143]:
# convert predicted K's to cluster means
nPred = predKs.shape[0]
pred_means = np.ndarray([nPred,nPCs]) # create data structure

# populate
for p in range(nPred):
    pred_means[p] = mus[predKs[p]] # with assigned cluster mean
print pred_means.shape


(233286L, 10L)


In [144]:
# rotate these pca-space means back to (standardized) artist space
Vred = V.T[:nPCs].T
pred_scaled = np.dot(Vred,pred_means.T).T

print pred_scaled.shape

MemoryError: 

In [93]:
# unstandardize
pred_fin = std_scale.inverse_transform(pred_scaled)

In [87]:
# first, if validation set: test error?
truvals = data_mat[:nValid]

MAE = np.sum(np.absolute(pred_fin - truvals))/pred_fin.size
print MAE

186.534199595


In [102]:
print pred_fin.shape

(233286L, 2000L)


In [103]:
# explore test data to see if this is doing the right thing
test_df = pd.read_csv(test_file)
test_df.head()

,Id,user,artist
0,1,306e19cce2522fa2d39ff5dfc870992100ec22d2,4ac4e32b-bd18-402e-adad-ae00e72f8d85
1,2,9450d351278df4938bdea4ed86aec940a4e927ac,1f574ab1-a46d-4586-9331-f0ded23e0411
2,3,801909d6955f59033c88595d3d7f8a6a5dcd53cc,3eb72791-6322-466b-87d3-24d74901eb2d
3,4,e3ed47445c127fbeff47fb58f6bbf2f3b4535d82,61604b45-8a91-4e33-a1b6-45d7b1fec4e5
4,5,a73f46652103f3a5f7429159310f6928f79644aa,5dfdca28-9ddc-4853-933c-8bc97d87beec


In [109]:
test_vals = test_df.values
a = test_vals[0][1]
b = test_vals[0][2]

print a, b
i = prof_idx[a]
print all_ids[i]
j = art_idx[b]
print all_arts[j]
print data_mat[i][j]
print pred_fin[i][j]

306e19cce2522fa2d39ff5dfc870992100ec22d2 4ac4e32b-bd18-402e-adad-ae00e72f8d85
306e19cce2522fa2d39ff5dfc870992100ec22d2
4ac4e32b-bd18-402e-adad-ae00e72f8d85
0.0
128.890206284


In [110]:
print pred_fin[i][j]

128.890206284


In [96]:
soln_file = 'Kmeans_Preds.csv'

In [97]:
# now: use to populate predicted values
# Write out test solutions.
with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open(soln_file, 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])

        for row in test_csv:
            id     = row[0]
            user   = row[1]
            artist = row[2]
            
            # find corresponding index
            i = prof_idx[user] # use dictionary to find i value
            j = art_idx[artist] # use other dictionary to find j

            soln_csv.writerow([id, pred_fin[i][j]])


In [98]:
# Compute the global median and per-user median.
plays_array  = []
user_medians = {}
for user, user_data in train_data.iteritems():
    user_plays = []
    for artist, plays in user_data.iteritems():
        plays_array.append(plays)
        user_plays.append(plays)

    user_medians[user] = np.median(np.array(user_plays))
global_median = np.median(np.array(plays_array))

In [125]:
soln_file = 'median_test.csv'

# Write out test solutions.
with open(test_file, 'r') as test_fh:
    test_csv = csv.reader(test_fh, delimiter=',', quotechar='"')
    next(test_csv, None)

    with open(soln_file, 'w') as soln_fh:
        soln_csv = csv.writer(soln_fh,
                              delimiter=',',
                              quotechar='"',
                              quoting=csv.QUOTE_MINIMAL)
        soln_csv.writerow(['Id', 'plays'])

        for row in test_csv:
            id     = row[0]
            user   = row[1]
            artist = row[2]

            if user in user_medians:
                soln_csv.writerow([id, user_medians[user]])
            else:
                print "User", id, "not in training data."
                soln_csv.writerow([id, global_median])
                
